In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import re, string, unicodedata
import os
import inflect
import numpy as np
import pandas as pd
import collections
import networkx as nx
import time
import pickle

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk import download
from nltk.corpus import stopwords


from sklearn.metrics.pairwise import cosine_similarity


%pip install stanza
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=573bb1a37a0bdb18498270bda3cd6a302a61406a4b8819445b6d320edc23f76a
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [ ]:
download('stopwords')
download('punkt')
stanza.download('es')
nlp = stanza.Pipeline('es')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2022-01-12 04:46:19 INFO: Downloading default packages for language: es (Spanish)...


2022-01-12 04:46:37 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-01-12 04:46:37 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2022-01-12 04:46:37 INFO: Use device: cpu
2022-01-12 04:46:37 INFO: Loading: tokenize
2022-01-12 04:46:37 INFO: Loading: mwt
2022-01-12 04:46:37 INFO: Loading: pos
2022-01-12 04:46:38 INFO: Loading: lemma
2022-01-12 04:46:38 INFO: Loading: depparse
2022-01-12 04:46:39 INFO: Loading: ner
2022-01-12 04:46:41 INFO: Done loading processors!


In [ ]:
#Convierto cada tweet del dataset en un vector de palabras
from nltk import word_tokenize
def ConvertirDataAnterior(corpusBase):
  textos = []
  for y in [corpusBase[i][0] for i in range(len(corpusBase))]:
      doc = word_tokenize(y)
      textos.append(doc)
  return textos

def ConvertirData(data):
  comments = []
  for line in data:
      comments.append(str(line).split(' '))
  return comments

In [ ]:
DataFinal = pd.read_excel('/content/gdrive/My Drive/ART_EMERGENCY/GLOVE_CODIGO/4DataFinalLemaFiltro.xlsx')

FUNCIONES

In [ ]:
def Knn(Matriz,tipo, k):
      labels = tipo
      pred_label = []
      etiquetar = []
      indice = 0
      for x in labels:
          if(indice != len(labels)-1):
            if(Matriz[indice]<=0.25): #Filtro que asigan directament 0 cuando el valor de comparacion sea cero o menor a cero
              etiquetar.append((Matriz[indice], 0))
            else:
              etiquetar.append((Matriz[indice], x))
          indice += 1
      etiquetar.sort(reverse=True)
      neighbors = etiquetar[:k]
      votes = []
      for neighbor in neighbors:
                votes.append(neighbor[1])
      counter = collections.Counter(votes) #Determina el que mas se repite
      pred_label.append(counter.most_common()[0][0])
      return pred_label[0]

MODELO GLOVE

In [ ]:
from gensim.models import Word2Vec

In [ ]:
    import numpy as np
    from sklearn.model_selection import train_test_split
    import pandas as pd
    from sklearn.metrics import accuracy_score

      
    def fit_Word2Vec(textos):
      model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_count= 1 , min_alpha=0.0007,
                 seed = 14)

      model.build_vocab(textos, progress_per=200)
      model.train(textos, total_examples = model.corpus_count, 
                  epochs=10, report_delay=1)

      return model

      

    def modeloWord2Vec(Tweet,word2vec_model,data_vector):

      if len(Tweet.split()) <= 2: #Se valida que el tweet preprocesado tenga un minino de palabras
        Tweet = 'falso'  #Se le asigna el texto 'vacio' el cual sera detectado como no emergencia

      eje_vectors =  np.mean([word2vec_model.wv[word] for word in Tweet.split() if word in word2vec_model.wv.vocab], axis=0)
      sim=[]

      for i in data_vector:
        similarity = (i @ eje_vectors) / (np.linalg.norm (i) * np.linalg.norm (eje_vectors)) 
        sim.append(similarity)
      
      return sim


    def predict_Word2Vec(Train, TrainOri, y_train, tweets):
      prediccion = []
      word2vec_model = fit_Word2Vec(Train)
      
      data_vector = []
      for comment in TrainOri:
        words_vectors = np.mean([word2vec_model.wv[word] for word in comment.split() if word in word2vec_model.wv.vocab], axis=0)
        # Aqui puedes promediar los vectores de las palabras o utilizar otra técnica para representar al comentario
        data_vector.append(words_vectors)

      for tweet in [tweets[i][0] for i in range(len(tweets))]:
        Similaridad= modeloWord2Vec(tweet,word2vec_model,data_vector)
        prediccion.append(k_predict_Word2Vec(Similaridad,y_train))
      return prediccion

    def k_predict_Word2Vec(Similaridad, y_train):
      k_1 = Knn(Similaridad,y_train, 1)
      k_3 = Knn(Similaridad,y_train, 3)
      k_5 = Knn(Similaridad,y_train, 5)
      k_7 = Knn(Similaridad,y_train, 7)
      k_9 = Knn(Similaridad,y_train, 9)
      k_11 = Knn(Similaridad,y_train, 11)
      return k_1, k_3, k_5, k_7, k_9, k_11 


In [ ]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

X = DataFinal['text']
y = DataFinal['tipo']

kf = KFold(n_splits=8, shuffle=True, random_state=2) 

In [ ]:
#MODELO Word2vec 

In [ ]:

from sklearn.metrics import confusion_matrix
nombre = []
accuracy = []
precision = []
recall = []
sensitivy = []
f1 = []
log = []

indice_fold=1



for rep in range(30):
  indice_fold=1
  for entrenamiento_indice, prueba_indice in kf.split(X):
    #print(entrenamiento_indice,prueba_indice)
    X_train, X_test = X[entrenamiento_indice],X[prueba_indice]
    y_train, y_test = y[entrenamiento_indice],y[prueba_indice]
    X_train = X_train.values.reshape([X_train.values.shape[0],1])
    X_test = X_test.values.reshape([X_test.values.shape[0],1]) 
    
    X_proce = ConvertirData(X[entrenamiento_indice]) 
    y_pred =  predict_Word2Vec(X_proce,X[entrenamiento_indice],y_train,X_test)
    y_pred = np.array(y_pred)
    a = [1,3,5,7,9,11]
    print(' COMPLETADO:')
    for i in range(6):
      name = 'WORD2VEC K'+str(a[i])+', FOLD '+str(indice_fold)+', REPETICION '+str(rep +1)
      nombre.append(name)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred[:,i]).ravel()
      accuracy.append(metrics.accuracy_score(y_test,y_pred[:,i]))
      precision.append(metrics.precision_score(y_test,y_pred[:,i]))
      f1.append(metrics.f1_score(y_test,y_pred[:,i]))
      log.append(metrics.log_loss(y_test,y_pred[:,i]))
      recall.append( tn / float(tn + fp))
      sensitivy.append(tp / float(tp + fn))
      print(' '+name)
      #print(accuracy, precision, recall, sensitivy)
    indice_fold = indice_fold + 1 

 COMPLETADO:
 WORD2VEC K1, FOLD 1, REPETICION 1
 WORD2VEC K3, FOLD 1, REPETICION 1
 WORD2VEC K5, FOLD 1, REPETICION 1
 WORD2VEC K7, FOLD 1, REPETICION 1
 WORD2VEC K9, FOLD 1, REPETICION 1
 WORD2VEC K11, FOLD 1, REPETICION 1
 COMPLETADO:
 WORD2VEC K1, FOLD 2, REPETICION 1
 WORD2VEC K3, FOLD 2, REPETICION 1
 WORD2VEC K5, FOLD 2, REPETICION 1
 WORD2VEC K7, FOLD 2, REPETICION 1
 WORD2VEC K9, FOLD 2, REPETICION 1
 WORD2VEC K11, FOLD 2, REPETICION 1
 COMPLETADO:
 WORD2VEC K1, FOLD 3, REPETICION 1
 WORD2VEC K3, FOLD 3, REPETICION 1
 WORD2VEC K5, FOLD 3, REPETICION 1
 WORD2VEC K7, FOLD 3, REPETICION 1
 WORD2VEC K9, FOLD 3, REPETICION 1
 WORD2VEC K11, FOLD 3, REPETICION 1
 COMPLETADO:
 WORD2VEC K1, FOLD 4, REPETICION 1
 WORD2VEC K3, FOLD 4, REPETICION 1
 WORD2VEC K5, FOLD 4, REPETICION 1
 WORD2VEC K7, FOLD 4, REPETICION 1
 WORD2VEC K9, FOLD 4, REPETICION 1
 WORD2VEC K11, FOLD 4, REPETICION 1
 COMPLETADO:
 WORD2VEC K1, FOLD 5, REPETICION 1
 WORD2VEC K3, FOLD 5, REPETICION 1
 WORD2VEC K5, FOLD 5,

In [ ]:
resultado2=[nombre,accuracy,precision,recall,sensitivy,f1,log]
Final2 = pd.DataFrame(resultado2)
Final2.T

,0,1,2,3,4,5,6
0,"WORD2VEC K1, FOLD 1, REPETICION 1",0.817365,0.802817,0.850267,0.77551,0.788927,6.308047
1,"WORD2VEC K3, FOLD 1, REPETICION 1",0.835329,0.823944,0.86631,0.795918,0.809689,5.687583
2,"WORD2VEC K5, FOLD 1, REPETICION 1",0.859281,0.857143,0.893048,0.816327,0.836237,4.860295
3,"WORD2VEC K7, FOLD 1, REPETICION 1",0.862275,0.858156,0.893048,0.823129,0.840278,4.756885
4,"WORD2VEC K9, FOLD 1, REPETICION 1",0.859281,0.862319,0.898396,0.809524,0.835088,4.860292
...,...,...,...,...,...,...,...
1435,"WORD2VEC K3, FOLD 8, REPETICION 30",0.852853,0.856209,0.874286,0.829114,0.842444,5.082335
1436,"WORD2VEC K5, FOLD 8, REPETICION 30",0.864865,0.869281,0.885714,0.841772,0.855305,4.66745
1437,"WORD2VEC K7, FOLD 8, REPETICION 30",0.870871,0.875817,0.891429,0.848101,0.861736,4.460008
1438,"WORD2VEC K9, FOLD 8, REPETICION 30",0.867868,0.885135,0.902857,0.829114,0.856209,4.563723


In [ ]:
Final2.T.to_excel('Pruebas_WORD2VEC.xlsx', index=False)
!cp Pruebas_WORD2VEC.xlsx '/content/gdrive/My Drive/ART_EMERGENCY/PRUEBAS/'